In [23]:
%env CUDA_VISIBLE_DEVICES=-1

env: CUDA_VISIBLE_DEVICES=-1


In [24]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD, RMSprop, Adagrad, Adam
from tensorflow import device

In [25]:
from numpy import load, stack
from itertools import cycle

def bachelor(file, base, batch=32):
    """Generate a batch of training pair of data from directory"""
    with device('/cpu:0'):
        names = open(file+".lst")
        datas = load(file+".npy")
        X = []; y = []; c = 0;
        
        while True:
            for name, data in cycle(zip(names, datas)):
                spec = load(f"{base}/{name.strip()}.npy")
                spec = spec.reshape(spec.shape + (1,))
                para = data.reshape((data.shape[0],))
                X.append(spec); y.append(para); c += 1
                if c == batch:
                    yield stack(X), stack(y)
                    X = []; y = []; c = 0;

## Model

In [26]:
NAME = "linear"

In [46]:
def new_model(name=NAME):
    """Model definition"""
    model = Sequential(name=name)
#    model.add(InputLayer((64,64,1), name="Input0"))

    model.add(Conv1D(8, 32, name="Conv0", input_shape=(4096,1)))
    model.add(LeakyReLU(name="LeReLU0"))
    model.add(MaxPooling1D(6, padding="same", name="Max0"))
    
    model.add(Conv1D(16, 16, name="Conv1"))
    model.add(LeakyReLU(name="LeReLU1"))
    model.add(MaxPooling1D(6, padding="same", name="Max1"))
    
    model.add(Conv1D(32, 8, name="Conv2"))
    model.add(LeakyReLU(name="LeReLU2"))
    model.add(MaxPooling1D(6, padding="same", name="Max2"))
    
    model.add(Flatten(name="Flat"))
    
    model.add(Dropout(0.2, name="Dropout"))    
    
    model.add(Dense(512, activation="tanh"))
    model.add(Dense(128, activation="tanh"))
    model.add(Dense(32, activation="tanh"))
    model.add(Dense(3, activation="linear"))
    return model

In [47]:
MODEL = new_model(NAME)
MODEL.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv0 (Conv1D)               (None, 4065, 8)           264       
_________________________________________________________________
LeReLU0 (LeakyReLU)          (None, 4065, 8)           0         
_________________________________________________________________
Max0 (MaxPooling1D)          (None, 678, 8)            0         
_________________________________________________________________
Conv1 (Conv1D)               (None, 663, 16)           2064      
_________________________________________________________________
LeReLU1 (LeakyReLU)          (None, 663, 16)           0         
_________________________________________________________________
Max1 (MaxPooling1D)          (None, 111, 16)           0         
_________________________________________________________________
Conv2 (Conv1D)               (None, 104, 32)           4128      
__________

In [48]:
MODEL.compile(optimizer="sgd", loss="mse", metrics=['accuracy'])

In [49]:
with device('/cpu:0'):
    MODEL.fit_generator(bachelor("train", "linear"),
                        steps_per_epoch=845, epochs=1,
                        validation_data=bachelor("tests", "linear"),
                        validation_steps=282)

Epoch 1/1
845/845 [==============================] - 36s 42ms/step - loss: 0.1640 - acc: 0.8682 - val_loss: 0.1116 - val_acc: 0.9070


In [50]:
MODEL.save(NAME+".h5")